In [ ]:
# 1. Importações
import pandas as pd

# Inicializar lista para armazenar DataFrames de cada ano
dfs = []

# Concatenar os ficheiros de 1992 a 2023
for year in range(1992, 2024):
    file = f"D:/projetos/Tenis ML-AI/data/tennis_atp/atp_matches_{year}.csv"
    year_data = pd.read_csv(file)
    dfs.append(year_data)

# Concatenar todos os DataFrames de uma vez só
all_data = pd.concat(dfs, axis=0, ignore_index=True)

# Visualizar primeiras linhas
all_data.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1992-339,Adelaide,Hard,32,A,19911230,1,101964,1.0,NaN,...,34.0,23.0,6.0,9.0,0.0,3.0,16.0,NaN,80.0,NaN
1,1992-339,Adelaide,Hard,32,A,19911230,2,101924,NaN,NaN,...,65.0,39.0,9.0,10.0,8.0,12.0,65.0,NaN,63.0,NaN
2,1992-339,Adelaide,Hard,32,A,19911230,3,101195,NaN,NaN,...,68.0,45.0,22.0,16.0,8.0,12.0,62.0,NaN,730.0,NaN
3,1992-339,Adelaide,Hard,32,A,19911230,4,101820,NaN,NaN,...,49.0,34.0,16.0,14.0,1.0,4.0,60.0,NaN,42.0,NaN
4,1992-339,Adelaide,Hard,32,A,19911230,5,100870,NaN,NaN,...,95.0,65.0,15.0,18.0,5.0,7.0,68.0,NaN,32.0,NaN


In [3]:
# 1. Manter só jogos com dados essenciais
critical_cols = [
    'tourney_id', 'tourney_name', 'surface', 'tourney_date',
    'winner_name', 'loser_name',
    'winner_age', 'loser_age',
    'winner_rank', 'loser_rank'
]
all_data_filtered = all_data.dropna(subset=critical_cols)

# 2. Depois filtra para manter apenas as colunas desejadas
final_cols = [
    'tourney_id','tourney_name','surface','draw_size','tourney_level','tourney_date','match_num',
    'winner_id','winner_seed','winner_entry','winner_name','winner_hand','winner_ht','winner_ioc','winner_age',
    'loser_id','loser_seed','loser_entry','loser_name','loser_hand','loser_ht','loser_ioc','loser_age',
    'score','best_of','round','minutes',
    'w_ace','w_df','w_svpt','w_1stIn','w_1stWon','w_2ndWon','w_SvGms','w_bpSaved','w_bpFaced',
    'l_ace','l_df','l_svpt','l_1stIn','l_1stWon','l_2ndWon','l_SvGms','l_bpSaved','l_bpFaced',
    'winner_rank','winner_rank_points','loser_rank','loser_rank_points'
]

# 3. Filtrar apenas essas colunas
all_data_filtered = all_data_filtered[final_cols].reset_index(drop=True)

print(f"Jogos válidos após filtro equilibrado: {all_data_filtered.shape[0]}")

Jogos válidos após filtro equilibrado: 98538


In [4]:
numeric_cols = ['draw_size', 'match_num', 'winner_ht', 'loser_ht', 'winner_age', 'loser_age', 'minutes',
                'w_ace','w_df','w_svpt','w_1stIn','w_1stWon','w_2ndWon','w_SvGms','w_bpSaved','w_bpFaced',
                'l_ace','l_df','l_svpt','l_1stIn','l_1stWon','l_2ndWon','l_SvGms','l_bpSaved','l_bpFaced',
                'winner_rank','winner_rank_points','loser_rank','loser_rank_points']

for col in numeric_cols:
    all_data_filtered[col] = pd.to_numeric(all_data_filtered[col], errors='coerce')

In [5]:
missing_values = all_data.isnull().sum()
missing_values[missing_values > 0].sort_values(ascending=False)

winner_entry          89152
loser_entry           81078
loser_seed            78905
winner_seed           60352
minutes               12546
l_bpFaced              9774
w_svpt                 9774
w_ace                  9774
w_df                   9774
w_1stWon               9774
w_1stIn                9774
w_2ndWon               9774
l_1stIn                9774
l_df                   9774
l_ace                  9774
w_bpFaced              9774
w_bpSaved              9774
l_svpt                 9774
l_bpSaved              9774
l_1stWon               9774
l_2ndWon               9774
l_SvGms                9773
w_SvGms                9773
loser_ht               4234
loser_rank_points      3206
loser_rank             2394
winner_ht              2114
winner_rank_points     1921
winner_rank            1105
surface                  53
loser_age                 7
winner_age                4
loser_hand                4
dtype: int64

In [13]:
# 1. Carregar o dataset de Elo
elo = pd.read_csv("D:/projetos/Tenis ML-AI/data/elo_ratings.csv", encoding="latin1")

# 2. Função de normalização melhorada (consistente com o que você já tem)
def normalize_name(name):
    if pd.isna(name):
        return ""
    name = str(name).strip().lower()
    # Remove caracteres especiais e pontuação
    replacements = {
        '-': ' ',
        '.': '',
        "'": '',
        'à': 'a', 'á': 'a', 'â': 'a', 'ã': 'a', 'ä': 'a',
        'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e',
        'ì': 'i', 'í': 'i', 'î': 'i', 'ï': 'i',
        'ò': 'o', 'ó': 'o', 'ô': 'o', 'õ': 'o', 'ö': 'o',
        'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u',
        'ç': 'c', 'ñ': 'n'
    }
    for old, new in replacements.items():
        name = name.replace(old, new)
    return name

# 3. Criar colunas com nomes normalizados em ambos datasets
all_data_filtered['winner_name_clean'] = all_data_filtered['winner_name'].apply(normalize_name)
all_data_filtered['loser_name_clean'] = all_data_filtered['loser_name'].apply(normalize_name)
elo['player_name_clean'] = elo['Player'].apply(normalize_name)

# 4. Verificar correspondência de jogadores
match_players = set(all_data_filtered['winner_name_clean']).union(set(all_data_filtered['loser_name_clean']))
elo_players = set(elo['player_name_clean'])

print(f"Total de jogadores únicos nas partidas: {len(match_players)}")
print(f"Total de jogadores únicos no Elo: {len(elo_players)}")
print(f"Jogadores em partidas mas não em Elo: {len(match_players - elo_players)}")
print(f"Jogadores em Elo mas não em partidas: {len(elo_players - match_players)}")

# 5. Criar dicionários de Elo ratings por superfície
# Primeiro, garantir que as colunas existem no dataset Elo
if 'hardcourtelorating' not in elo.columns:
    elo['hardcourtelorating'] = elo['HardRaw']  # ou outra coluna equivalente
if 'claycourtelorating' not in elo.columns:
    elo['claycourtelorating'] = elo['ClayRaw']  # ou outra coluna equivalente
if 'grasscourtelorating' not in elo.columns:
    elo['grasscourtelorating'] = elo['GrassRaw']  # ou outra coluna equivalente

# Criar os dicionários
hard_elo = elo.set_index('player_name_clean')['hardcourtelorating'].to_dict()
clay_elo = elo.set_index('player_name_clean')['claycourtelorating'].to_dict()
grass_elo = elo.set_index('player_name_clean')['grasscourtelorating'].to_dict()

# 6. Função para obter Elo por superfície com tratamento robusto
def get_elo(player_name, surface):
    name = normalize_name(player_name)
    if pd.isna(name) or name == "":
        return float('nan')
    
    try:
        if surface == 'Hard':
            return hard_elo.get(name, float('nan'))
        elif surface == 'Clay':
            return clay_elo.get(name, float('nan'))
        elif surface == 'Grass':
            return grass_elo.get(name, float('nan'))
        else:
            return float('nan')
    except:
        return float('nan')

# 7. Adicionar Elo ao dataset principal
all_data_filtered['winner_elo'] = all_data_filtered.apply(
    lambda row: get_elo(row['winner_name'], row['surface']), axis=1)
all_data_filtered['loser_elo'] = all_data_filtered.apply(
    lambda row: get_elo(row['loser_name'], row['surface']), axis=1)

# 8. Calcular diferenças de Elo e outras features
all_data_filtered['elo_diff'] = all_data_filtered['winner_elo'] - all_data_filtered['loser_elo']
all_data_filtered['height_diff'] = all_data_filtered['winner_ht'] - all_data_filtered['loser_ht']
all_data_filtered['age_diff'] = all_data_filtered['winner_age'] - all_data_filtered['loser_age']
all_data_filtered['rank_diff'] = all_data_filtered['winner_rank'] - all_data_filtered['loser_rank']
all_data_filtered['rank_points_diff'] = all_data_filtered['winner_rank_points'] - all_data_filtered['loser_rank_points']

# 9. Verificar os resultados
print("\nAmostra dos dados com Elo:")
print(all_data_filtered[['winner_name', 'loser_name', 'surface', 'winner_elo', 'loser_elo', 'elo_diff']].head())

# 10. Estatísticas de valores ausentes
print("\nValores ausentes no Elo:")
print(f"Vencedores sem Elo: {all_data_filtered['winner_elo'].isna().sum()}")
print(f"Perdedores sem Elo: {all_data_filtered['loser_elo'].isna().sum()}")


Total de jogadores únicos nas partidas: 2684
Total de jogadores únicos no Elo: 783
Jogadores em partidas mas não em Elo: 2312
Jogadores em Elo mas não em partidas: 411

Amostra dos dados com Elo:
            winner_name         loser_name surface  winner_elo  loser_elo  \
0      Goran Ivanisevic      Nicklas Kulti    Hard         NaN        NaN   
1   Stefano Pescosolido        Jimmy Arias    Hard         NaN        NaN   
2         Amos Mansdorf        Grant Doyle    Hard         NaN        NaN   
3           Marc Rosset  Cristiano Caratti    Hard         NaN        NaN   
4  Christo Van Rensburg     Javier Sanchez    Hard         NaN        NaN   

   elo_diff  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  

Valores ausentes no Elo:
Vencedores sem Elo: 67169
Perdedores sem Elo: 70679


In [11]:
normalize_name("Goran Ivanisevic") in elo['player_name_clean'].tolist()

False

In [12]:
all_data_filtered.to_csv("D:/projetos/Tenis ML-AI/notebooks/atp_matches_1992_2023.csv", index=False)